<a href="https://colab.research.google.com/github/eddieguo-1128/honors/blob/main/Thesis_Code_Part4_VisualBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Requirements
!pip install nltk
!pip install transformers
!pip install shutup
!pip install pytorch-lightning
!pip install torchMetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 45.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import torch
import torch.nn as nn
import pytorch_lightning as pl
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader, random_split
from pylab import rcParams
from tqdm.auto import tqdm
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
#from torchmetrics.functional import f1, accuracy
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from matplotlib import rc

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
pl.seed_everything(RANDOM_SEED)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# Connect to drive and acquire data from Part 1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/SP23/Honors Thesis/DATA H195B/repo/honors

/content/drive/My Drive/SP23/Honors Thesis/DATA H195B/repo/honors


In [ ]:
reports_df = pd.read_csv('reports_df.csv',index_col=0)
images_df = pd.read_csv('images_df.csv',index_col=0)
combined = reports_df.merge(images_df,on='id')
combined = combined.drop(columns=['label_y'])
combined

,id,findings,impression,full-text,label_x,index,image_id
0,162,Heart size normal. Lungs are clear. XXXX are n...,Normal chest,Heart size normal. Lungs are clear. XXXX are n...,normal,0,NLMCXR_png/CXR162_IM-0401-1001.png
1,162,Heart size normal. Lungs are clear. XXXX are n...,Normal chest,Heart size normal. Lungs are clear. XXXX are n...,normal,0,NLMCXR_png/CXR162_IM-0401-2001.png
2,1390,The heart size and pulmonary vascularity appea...,No evidence of active disease.,The heart size and pulmonary vascularity appea...,abnormal,1,NLMCXR_png/CXR1390_IM-0249-1001.png
3,1390,The heart size and pulmonary vascularity appea...,No evidence of active disease.,The heart size and pulmonary vascularity appea...,abnormal,1,NLMCXR_png/CXR1390_IM-0249-2001.png
4,604,The heart size and pulmonary vascularity appea...,1. No evidence of active disease.,The heart size and pulmonary vascularity appea...,abnormal,2,NLMCXR_png/CXR604_IM-2193-1001.png
...,...,...,...,...,...,...,...
7569,85,There are scattered calcified granulomas. No f...,No evidence of active disease.,There are scattered calcified granulomas. No f...,abnormal,3953,NLMCXR_png/CXR85_IM-2372-1001.png
7570,85,There are scattered calcified granulomas. No f...,No evidence of active disease.,There are scattered calcified granulomas. No f...,abnormal,3953,NLMCXR_png/CXR85_IM-2372-2001.png
7571,1389,NaN,Heart size is normal. Lungs are clear. Overexp...,Heart size is normal. Lungs are clear. Overex...,abnormal,3954,NLMCXR_png/CXR1389_IM-0247-1001.png
7572,1389,NaN,Heart size is normal. Lungs are clear. Overexp...,Heart size is normal. Lungs are clear. Overex...,abnormal,3954,NLMCXR_png/CXR1389_IM-0247-2001.png


In [ ]:
!gdown --id 1mhT-osDT15obu8FVsgfDAoDd82IY-XQB # visual_embeds_mami.pkl

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1mhT-osDT15obu8FVsgfDAoDd82IY-XQB 

